In [1]:
%cd ..
%pwd

/home/markus/Documents/regression-prediction


'/home/markus/Documents/regression-prediction'

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import time
url = "https://treeherder.mozilla.org/api/performance/alertsummary/"
alertsummaries = []
i = 0
while True:
    i += 1
    print(f"{i}. GET {url}", end="\r")
    r = requests.get(url, headers={'User-Agent': 'basic'})
    j = r.json()
    alertsummaries += j['results']
    url = j['next']
    if url is None:
        break
    time.sleep(0.5)

In [ ]:
import datetime
date = datetime.datetime.now().strftime('%Y-%m-%d')
write_json_to_file(alertsummaries, f'data/labeling/alerts_{date}.json')

In [4]:
from src.utils import *
from src.labeling import *
from src.repo_miner import get_commit_log

In [5]:
alertsummaries = read_data_from_json('data/labeling/alerts_2022-03-01.json')
alertsummaries2 = read_data_from_json('data/labeling/alerts_2021-09-13.json')

In [6]:
alert_ids = {alert['id'] for alert in alertsummaries}
alert_ids2 = {alert['id'] for alert in alertsummaries2}

In [7]:
len(alert_ids), len(alert_ids2), len(alert_ids.intersection(alert_ids2))

(4592, 4460, 2516)

In [8]:
for alert in alertsummaries2:
    if alert['id'] not in alert_ids:
        alertsummaries.append(alert)

In [9]:
all_commits = get_all_commits()
hg_to_git, git_to_hg = get_hg_git_mapping()
commit_log = get_commit_log('data/repo_miner/commit_log.csv')

Get commits: 581679it [01:37, 5966.38it/s] 


Get hg to git mapping ... Done.


In [10]:
szz_labeling = pd.read_csv(f'data/labeling/fixed_defect_szz.csv')
fix_bug_ids_by_kind, _ = get_defects_and_fixes() # TODO: rerun and compare (fix invalid bugs)
fix_and_introducers = read_data_from_json(f'data/labeling/fixed_defect_szz/results/fix_and_introducers_pairs.json')

Get defects and fixes: 206038it [00:32, 6263.54it/s] 

Found
46417 regression
1337 security
961 performance
122 memory
3348 crash
39 power
fix bug ids.


In [11]:
for pair in fix_and_introducers:
    pair[0] = git_to_hg[pair[0]]
    pair[1] = git_to_hg[pair[1]]

In [12]:
len(fix_and_introducers)

2495

In [13]:
fix_and_introducers_df = pd.DataFrame(fix_and_introducers, columns=['fix', 'introducer'])
fix_and_introducers_df

,fix,introducer
0,b6113f461cb8bc50be357b7e84793c11f7a5c179,a6da1179a589e207da62749f5966f30024f7f0c9
1,714d667c7f120e7218ca08fb0f29508f4c81cc92,21ce1e95d697cbc1bdaa45d124e55642ff5b2fd3
2,714d667c7f120e7218ca08fb0f29508f4c81cc92,6daa67c0cae427dc8e514d5f1beea9f71ee4d8ef
3,714d667c7f120e7218ca08fb0f29508f4c81cc92,e53cd28c6cf360d7d9aa3a35051582112e608df1
4,714d667c7f120e7218ca08fb0f29508f4c81cc92,be7a5a989cb3dbabce6a1bb78aad2d05baa3589a
...,...,...
2490,8a0c0ad101aba2f397fa58932315004aa33de2d9,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5
2491,8a0c0ad101aba2f397fa58932315004aa33de2d9,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5
2492,e3d87330819774902687ba23d1a76b5f46daf26d,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5
2493,40b67d5f6d17f1e206d283d20bc35da5272fee38,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5


In [14]:
# add bug id of fix
fix_and_introducers_df = fix_and_introducers_df.merge(all_commits[['revision', 'bug_id']], left_on='fix', right_on='revision').drop('revision', axis=1)
fix_and_introducers_df

,fix,introducer,bug_id
0,b6113f461cb8bc50be357b7e84793c11f7a5c179,a6da1179a589e207da62749f5966f30024f7f0c9,1693027
1,714d667c7f120e7218ca08fb0f29508f4c81cc92,21ce1e95d697cbc1bdaa45d124e55642ff5b2fd3,1582797
2,714d667c7f120e7218ca08fb0f29508f4c81cc92,6daa67c0cae427dc8e514d5f1beea9f71ee4d8ef,1582797
3,714d667c7f120e7218ca08fb0f29508f4c81cc92,e53cd28c6cf360d7d9aa3a35051582112e608df1,1582797
4,714d667c7f120e7218ca08fb0f29508f4c81cc92,be7a5a989cb3dbabce6a1bb78aad2d05baa3589a,1582797
...,...,...,...
2490,59a0ddad5c28302d22089be78e59f2a058898842,17190bff616b3cbaca692825c64e6650a1de9562,1681030
2491,a5e88ddf8867e24f09fbb0bf0491d107cf68028d,b4197fcbd41190d40e4eec3ab4c2e270d80c7424,1665196
2492,e3d87330819774902687ba23d1a76b5f46daf26d,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5,1596709
2493,40b67d5f6d17f1e206d283d20bc35da5272fee38,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5,1596709


In [15]:
fix_and_introducers_df[fix_and_introducers_df['introducer'] == '9ac290ec5884fd52bb6c16e9794da5b42f211cbb']

,fix,introducer,bug_id
2108,5235f051d95235e351d59f61d5ce19cf0cb9d3f7,9ac290ec5884fd52bb6c16e9794da5b42f211cbb,1722487


In [14]:
introducers = set(fix_and_introducers_df['introducer'])
len(set(fix_and_introducers_df['fix'])), len(introducers)

(431, 1544)

In [15]:
print_labeling_stats(szz_labeling, 'performance')

positive labels: 589 0.59%, negative labels: 99105 99.41%


In [16]:
selected_introducers = set(szz_labeling.loc[szz_labeling['performance']==1, 'revision'])
len(selected_introducers)

589

In [17]:
not_selected_introducers = list(introducers.difference(selected_introducers))
commit_log.loc[not_selected_introducers].sort_values('id').tail(25)

,id,revision,date,author,description,main,short
index,,,,,,,
29d310d368efef7ae997f69dc979e7fcb6f6bfa9,480492,29d310d368efef7ae997f69dc979e7fcb6f6bfa9,2019-06-28 07:08:00+00:00,Matt Woodrow <mwoodrow@mozilla.com>,Bug 1561178 - Initialize gfxVars in the RDD pr...,True,29d310d368ef
133ac0d11428079ceaeceab4dda050b4ba240e3b,480620,133ac0d11428079ceaeceab4dda050b4ba240e3b,2019-06-28 21:19:00+00:00,"Kearwood ""Kip"" Gilbert <kgilbert@mozilla.com>",Bug 1466702 - Refactor / reimplement gfxVRPupp...,True,133ac0d11428
1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5,480624,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5,2019-06-28 23:39:00+00:00,Narcis Beleuzu <nbeleuzu@mozilla.com>,Backed out 2 changesets (bug 1553705) for caus...,True,1ddc47f73701
aa12d9e8a4f1cad6b8057ef69fa4b3273c3c3fab,481366,aa12d9e8a4f1cad6b8057ef69fa4b3273c3c3fab,2019-07-05 07:48:00+00:00,Victor Porof <vporof@mozilla.com>,"Bug 1561435 - Format browser/base/, a=automati...",True,aa12d9e8a4f1
4381f16ec9948a70802d7cc28cd021b094c7ba74,481367,4381f16ec9948a70802d7cc28cd021b094c7ba74,2019-07-05 07:53:00+00:00,Victor Porof <vporof@mozilla.com>,"Bug 1561435 - Format browser/components/, a=au...",True,4381f16ec994
e8225f3e114bf1fc01f9e574cc41eb2df4e314c6,481369,e8225f3e114bf1fc01f9e574cc41eb2df4e314c6,2019-07-05 07:55:00+00:00,Victor Porof <vporof@mozilla.com>,"Bug 1561435 - Format browser/modules/, a=autom...",True,e8225f3e114b
f3b422727b92f4933ca41fa7a4013d829a98240d,481377,f3b422727b92f4933ca41fa7a4013d829a98240d,2019-07-05 08:44:00+00:00,Victor Porof <vporof@mozilla.com>,"Bug 1561435 - Format dom/, a=automatic-formatt...",True,f3b422727b92
599bcfdc8eee2a5c6df4bfebdf5dd94c31d6b501,481382,599bcfdc8eee2a5c6df4bfebdf5dd94c31d6b501,2019-07-05 08:46:00+00:00,Victor Porof <vporof@mozilla.com>,"Bug 1561435 - Format extensions/, a=automatic-...",True,599bcfdc8eee
a85aec1aa92d1e067a5813a662175d4a76ce50a8,481392,a85aec1aa92d1e067a5813a662175d4a76ce50a8,2019-07-05 08:50:00+00:00,Victor Porof <vporof@mozilla.com>,"Bug 1561435 - Format layout/, a=automatic-form...",True,a85aec1aa92d


In [18]:
# fix_bug_ids_by_kind['performance'] are bug numbers for which we tried to find an introducer
alertsummaries_considered = [alert for alert in alertsummaries if alert['bug_number'] in fix_bug_ids_by_kind['performance']]

# total number of bug ids for which an alert exists and for which we tried to find an introducer
N = len({alert['bug_number'] for alert in alertsummaries_considered})

len(alertsummaries_considered), N

(297, 192)

In [19]:
# bug ids of fixes for which we actually found an introducer
bug_ids_found = set(fix_and_introducers_df['bug_id'])
assert len(bug_ids_found.difference(set(fix_bug_ids_by_kind['performance']))) == 0

# corresponding alerts
alertsummaries_found = [alert for alert in alertsummaries_considered if alert['bug_number'] in bug_ids_found]

# all bug ids for which an alert exists and for which we found an introducer
alert_bug_ids = {alert['bug_number'] for alert in alertsummaries_found}

TP_plus_FP = len(alert_bug_ids)

len(alertsummaries_found), TP_plus_FP # *bug numbers*

(144, 87)

In [20]:
# number of bug ids for which we tried to find an introducer but did not
# lower bound on false negative *commits*
N - TP_plus_FP

105

In [21]:
TP = 0
FP = 0

for bug_id in alert_bug_ids:
    introducers_for_bug = fix_and_introducers_df.loc[fix_and_introducers_df['bug_id'] == bug_id, 'introducer']
    introducers_for_bug = set(introducers_for_bug) # found introducers by SZZ 
    assert len(introducers_for_bug) > 0
    
    all_candidate_revisions = set()
    for alert in alertsummaries_considered: # can use all alertsummaries here TODO: check
        if alert['bug_number'] == bug_id:
            try:
                if alert['prev_push_revision'] == alert['revision']:
                    # this is sometimes the case, we just take the single commit
                    candidate_revisions = set(commit_log.loc[alert['prev_push_revision']:alert['revision'], 'revision'])
                else:
                    # performance change detected in interval (alert['prev_push_revision'], alert['revision']]
                    # exclude prev_push_revision commit TODO: check
                    candidate_revisions = set(commit_log.loc[alert['prev_push_revision']:alert['revision'], 'revision'].iloc[1:])
            except KeyError:
                pass
            all_candidate_revisions = all_candidate_revisions.union(candidate_revisions)

    TP += len(introducers_for_bug.intersection(candidate_revisions)) # number of found introducers in candidate revisions
    FP += len(introducers_for_bug.difference(candidate_revisions))

print(f'{TP=}, {FP=}, {TP+FP=}')
            

TP=38, FP=691, TP+FP=729


In [22]:
_, _, _, bugbug_fix_to_regressor = get_bugbug_regressors_and_fixes()


Get regressors and fixes: 206038it [00:53, 3860.44it/s]


Found
9562 regression
874 performance
1214 crash
281 security
32 memory
9 power
regressor bug ids,

and

15334 regression
965 performance
1577 crash
299 security
31 memory
9 power
fix bug ids.


In [23]:
bugbug_fixes = set(bugbug_fix_to_regressor.keys())
len(bugbug_fixes)

15334

In [24]:
bug_ids_considered = bugbug_fixes.intersection(set(fix_bug_ids_by_kind['performance']))
N = len(bug_ids_considered)
N

371

In [25]:
# bug ids for which we actually found an introducer
bug_ids_found = set(fix_and_introducers_df['bug_id'])
assert len(bug_ids_found.difference(set(fix_bug_ids_by_kind['performance']))) == 0

regressed_by_bug_ids = bug_ids_considered.intersection(bug_ids_found)
TP_plus_FP = len(regressed_by_bug_ids)
TP_plus_FP # *bug numbers*

164

In [26]:
# number of bug ids for which we tried to find an introducer but did not
# lower bound on false negative *commits*
N - TP_plus_FP

207

In [28]:
TP = 0
FP = 0
TP_commits = set()

for bug_id in regressed_by_bug_ids:
    introducers_for_bug = fix_and_introducers_df.loc[fix_and_introducers_df['bug_id'] == bug_id, 'introducer']
    introducers_for_bug = set(introducers_for_bug) # found introducers by SZZ 
    assert len(introducers_for_bug) > 0
    
    all_candidate_revisions = set()
    for regressors_bug_id in bugbug_fix_to_regressor[bug_id]:
        candidate_revisions = set(all_commits.loc[all_commits['bug_id'] == regressors_bug_id, 'revision'])
        all_candidate_revisions = all_candidate_revisions.union(candidate_revisions)

    tp_commits = introducers_for_bug.intersection(candidate_revisions)
    TP_commits = TP_commits.union(tp_commits)
    TP += len(tp_commits)
    FP += len(introducers_for_bug.difference(candidate_revisions))

print(f'{TP=}, {FP=}, {TP+FP=}')

TP=134, FP=576, TP+FP=710


In [32]:
write_json_to_file(list(TP_commits), 'experiments/results/szz_regressed_by_tp.json')

In [83]:
resolutions = set()
status = set()
types = set()
my_bug = None
with open('data/bugbug/bugs.json', encoding="utf-8") as f:
    for line in tqdm(f, desc='Get defects and fixes'):
        bug = json.loads(line)
        if bug['id'] == 1717171:
            my_bug = bug
        resolutions.add(bug['resolution'])
        status.add(bug['status'])
        types.add(bug['type'])

Get defects and fixes: 206038it [00:24, 8383.67it/s] 


In [84]:
resolutions

{'',
 'DUPLICATE',
 'EXPIRED',
 'FIXED',
 'INACTIVE',
 'INCOMPLETE',
 'INVALID',
 'MOVED',
 'WONTFIX',
 'WORKSFORME'}

In [85]:
status

{'ASSIGNED',
 'CLOSED',
 'NEW',
 'REOPENED',
 'RESOLVED',
 'UNCONFIRMED',
 'VERIFIED'}

In [86]:
types

{'--', 'defect', 'enhancement', 'task'}

In [109]:
selected_commits = get_selected_commits()

commit_log = get_commit_log('data/repo_miner/commit_log.csv')

Get commits: 581679it [04:44, 2041.99it/s]


In [105]:
len(selected_commits)

99694

In [108]:
selected_commits['date']

index
1582b3ffe71226af68979a58b5852afa49ff939b   2019-07-01 09:51:59
199bcb54e5122b5e7ac2476c90288b2169db2013   2019-07-01 09:51:59
7d964bfcda0e0aded01a4fb05a8405b9cb61bafd   2019-07-01 09:51:59
696aa5fa31ea5c39cee412a6cecfd4af19f845fd   2019-07-01 09:51:59
da65877c0fec3c21af44c542bada87ea8e8c6eea   2019-07-01 09:51:59
                                                   ...        
2e03fdd340bda8085be8e6dc6e5277f09aa8a171   2021-10-30 21:27:31
f398c3d4e21385a4b57ff67609fecca718a21416   2021-10-31 09:54:03
6e81bb1b5c7b19bff22b847d72d814cb1c7a63fa   2021-10-31 09:54:03
213bd1d0fa05af57af2c06f2bb6a42dfd75be970   2021-10-31 21:39:49
4f57142ccd08255c6e87390db6772db94a1033c4   2021-10-31 21:39:49
Name: date, Length: 99694, dtype: datetime64[ns]